In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Translation Evaluation with reference

## Install Vertex AI SDK for Python

In [1]:
!pip install --upgrade --quiet google-cloud-aiplatform[evaluation]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.8 MB/s eta 0:00:00


## Authenticate to access GCP

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

    from google.colab import drive
    drive.mount('/content/drive')

!gcloud config set project ai-hangsik

Mounted at /content/drive
Updated property [core/project].


## Initialize Vertex AI and Gemini model

In [3]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL = "gemini-1.5-pro"

import base64
import vertexai
from vertexai.generative_models import GenerativeModel

import pandas as pd
pd.options.display.max_colwidth = 300


vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel(MODEL)

## Pointwise evaluation

### Test dataset

In [4]:
# Pointwise 검사를 위한 데이터셋 준비
# Pointwise는 단일 모델의 결과에 대한 평가입니다.
data = [
    ["This account is not protected with a two-factor authentication.",
     "2차 인증이 되지 않은 계정입니다.",
     "이중 인증으로 보호되지 않은 계정입니다."],
    ["Even the longest journey begins with a single step.",
    "천 리 길도 한 걸음부터.",
    "가장 먼 여정도 한 걸음에서 시작됩니다."],
    ["Even the longest journey begins with a single step.",
    "Лиха беда начало.",
    "Даже самая длинная дорога продолжается с одного шага."]
]
pointwise_df = pd.DataFrame(data, columns=['prompt', 'reference', 'response'])
pointwise_df

,prompt,reference,response
0,This account is not protected with a two-factor authentication.,2차 인증이 되지 않은 계정입니다.,이중 인증으로 보호되지 않은 계정입니다.
1,Even the longest journey begins with a single step.,천 리 길도 한 걸음부터.,가장 먼 여정도 한 걸음에서 시작됩니다.
2,Even the longest journey begins with a single step.,Лиха беда начало.,Даже самая длинная дорога продолжается с одного шага.


In [5]:
from vertexai.evaluation import EvalTask, PointwiseMetric, PairwiseMetric
import vertexai.evaluation.metrics.pointwise_metric as pointwise_metric

pointwise_ground_truth_metric_prompt = """

# Instruction
당신은 전문적인 번역 품질 평가자 입니다. 당신은 AI 모델이 번역한 내용의 품질을 평가해야 합니다.
우리는 당신에게 'Source, AI translation, Ground Truth'를 제공할 것입니다.
업무를 수행하기 위해 'Source, AI translation, Ground Truth'을 신중하게 읽고, 아래의 'Evaluation' 섹션에 정의된 'Evaluation criteria'에 근거하여 평가해야 합니다.
당신은 'Rating rubric'과 'Evaluation steps'에 기반하여 평가해야 합니다. 평가의 이유에 대해 단계별로 '한글'로 설명하고 'Rating rubric'의 순위만 선택해야 합니다.

# Evaluation
## Metric Definition
번역이 'Ground Truth'와 동일하게 번역되었는지, 그렇지 않으면 의미적으로 유사하고 간결, 명확, 캐주얼하게 작성되었는지 전반적으로 확인합니다.

## Evaluation criteria
예시 일치성: 예시로 제공되는 'Ground Truth'와 얼마나 일치하는지 확인합니다.
지시를 잘 따르는지: 'AI translation'이 'Evaluation Definition'을 잘 따라서 수행됐는지 확인합니다.

## Rating rubric
5: (매우 좋음). 'Ground Truth'와 글자수 등 표현이 정확하게 일치함
4: (좋음). 'Ground Truth'와 어순 외 차이 없음
3: (괜찮음). 속담이나 관용적 표현을 사용하지 못했지만 이해하는데 큰 문제 없음
2: (나쁨). 이해하기가 어려움
1: (매우 나쁨). 번역결과가 'Ground Truth'와 매우 다름

## Evaluation steps
STEP 1: 'Ground Truth'을 참고하여 'AI translation'이 번역한 내용을 비교합니다.
STEP 2: 평가 규칙에 따라 점수를 부여합니다.

# Source, AI translation, Ground Truth
### Source
{prompt}

### Ground Truth
{reference}

### AI translation
{response}
"""

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.evaluation.PointwiseMetric

pointwise_ground_truth_text_quality = PointwiseMetric(
    metric="pointwise_ground_truth_quality",
    metric_prompt_template=pointwise_ground_truth_metric_prompt,
)
#For MetricX, require source field
pointwise_df['source'] = pointwise_df['prompt']

eval_task = EvalTask(dataset=pointwise_df,
                    metrics=[pointwise_ground_truth_text_quality, "bleu",  pointwise_metric.MetricX(), pointwise_metric.Comet()],
                    experiment="pointwise-eval")
result_pointwise = eval_task.evaluate()

#comet, higher is better (0-1)
#metricx, lower is better (0-25)
result_pointwise.metrics_table

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/pointwise-eval-516a073e-ca85-4e23-951d-a94148180d23 to Experiment: pointwise-eval


INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 12 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 12/12 [00:14<00:00,  1.22s/it]
INFO:vertexai.evaluation._evaluation:All 12 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:14.615197324000007 seconds


,prompt,reference,response,source,pointwise_ground_truth_quality/explanation,pointwise_ground_truth_quality/score,bleu/score,metricx/score,comet/score
0,This account is not protected with a two-factor authentication.,2차 인증이 되지 않은 계정입니다.,이중 인증으로 보호되지 않은 계정입니다.,This account is not protected with a two-factor authentication.,"STEP 1: AI 모델은 ""This account is not protected with a two-factor authentication.""을 ""이중 인증으로 보호되지 않은 계정입니다.""로 번역했습니다. Ground Truth는 ""2차 인증이 되지 않은 계정입니다.""입니다.\nSTEP 2: AI 번역은 ""이중 인증""이라는 표현을 사용했지만, Ground Truth는 ""2차 인증""이라는 표현을 사용했습니다. 두 표현 모두 비슷한 의미를 가지고 있지만, Ground Truth와 완전히 일치하지는 않습니다. 따라서 4점으로 평...",4.0,0.302138,0.961893,0.915181
1,Even the longest journey begins with a single step.,천 리 길도 한 걸음부터.,가장 먼 여정도 한 걸음에서 시작됩니다.,Even the longest journey begins with a single step.,"STEP 1: 'Ground Truth'는 '천 리 길도 한 걸음부터'로, 'AI translation'은 '가장 먼 여정도 한 걸음에서 시작됩니다.'입니다.\nSTEP 2: 'AI translation'은 'Ground Truth'와 의미는 유사하지만, 속담이나 관용적 표현을 사용하지는 못했습니다. 'Ground Truth'와 'AI translation'은 의미적으로는 유사하지만, 'Ground Truth'는 속담을 사용하여 간결하게 표현했고 'AI translation'은 직역체에 가깝습니다. 따라서 '괜찮음'으로 평가...",3.0,0.078099,0.964249,0.925086
2,Even the longest journey begins with a single step.,Лиха беда начало.,Даже самая длинная дорога продолжается с одного шага.,Even the longest journey begins with a single step.,"STEP 1: AI가 번역한 내용이 Ground Truth와 완전히 다릅니다. Ground Truth는 ""시작이 어려울 뿐이다.""라는 의미의 속담으로, AI는 ""가장 긴 길도 한 걸음부터 시작된다.""라고 직역했습니다.\nSTEP 2: AI 번역은 Ground Truth와 매우 다르기 때문에 1점으로 평가합니다.",1.0,0.047677,5.247432,0.568028


## Pairwise evaluation

In [6]:
# Pairwise 검사를 위한 데이터셋 준비
# Pairwise 는 2개 모델의 성능을 비교하기 위한 방법입니다.
#prompt(Source), reference, response (Response B), baseline_model_response (Response A)
data = [
    ["This account is not protected with a two-factor authentication.",
     "2차 인증이 되지 않은 계정입니다.",
     "이중 인증으로 보호되지 않은 계정입니다.",
     "이 계정은 이중 인증으로 안전하게 보호되지 않습니다."],
    ["Even the longest journey begins with a single step.",
    "천 리 길도 한 걸음부터.",
    "가장 먼 여정도 한 걸음에서 시작됩니다.",
    "가장 긴 여행조차도 한 걸음부터 시작됩니다."],
    ["Even the longest journey begins with a single step.",
    "Лиха беда начало.",
    "Даже самая длинная дорога продолжается с одного шага.",
    "Даже самый долгий путь начинается с первого шага."]
]
pairwise_df = pd.DataFrame(data, columns=['prompt', 'reference', 'response', 'baseline_model_response'])
pairwise_df

,prompt,reference,response,baseline_model_response
0,This account is not protected with a two-factor authentication.,2차 인증이 되지 않은 계정입니다.,이중 인증으로 보호되지 않은 계정입니다.,이 계정은 이중 인증으로 안전하게 보호되지 않습니다.
1,Even the longest journey begins with a single step.,천 리 길도 한 걸음부터.,가장 먼 여정도 한 걸음에서 시작됩니다.,가장 긴 여행조차도 한 걸음부터 시작됩니다.
2,Even the longest journey begins with a single step.,Лиха беда начало.,Даже самая длинная дорога продолжается с одного шага.,Даже самый долгий путь начинается с первого шага.


In [8]:
pairwise_model_compare_metric_prompt = """
# Instruction
당신은 전문적인 번역 품질 평가자 입니다. 당신은 2개의 AI 모델이 번역한 내용의 품질을 평가해야 합니다.
우리는 당신에게 'Source, Response A, Response B, Ground Truth'를 제공할 것입니다.
업무를 수행하기 위해 'Source, Response A, Response B, Ground Truth'을 신중하게 읽고, 아래의 '평가' 섹션에 정의된 '평가항목'에 근거하여 평가해야 합니다.
당신은 'Evaluation rule'과 'Evaluation steps'에 기반하여 평가해야 합니다. 평가의 이유에 대해 단계별로 '한글'로 설명하고 'Evaluation rule'의 순위만 선택해야 합니다.

# Evaluation
## Metric Definition
번역이 'Ground Truth'와 동일하게 번역되었는지, 그렇지 않으면 의미적으로 유사하고 간결, 명확, 캐주얼하게 작성되었는지 전반적으로 확인합니다.

## Evaluation criteria
예시 일치성: 예시로 제공되는 'Ground Truth'와 얼마나 일치하는지 확인합니다.
지시를 잘 따르는지: 'AI translation'이 'Evaluation Definition'을 잘 따라서 수행됐는지 확인합니다.

## Rating rubric
STEP 1: Analyze Response A based on all the Criteria.
STEP 2: Analyze Response B based on all the Criteria.
STEP 3: Compare the overall performance of Response A and Response B based on your analyses and assessment.
STEP 4: Output your preference of "A", "SAME" or "B" to the pairwise_choice field according to the Rating Rubric.
STEP 5: Output your assessment reasoning in the explanation field.

# Source, Response A, Response B, Ground Truth
### Source
{prompt}

### Ground Truth
{reference}

### Response A
{baseline_model_response}

### Response B
{response}
"""

pairwise_ground_truth_text_quality = PairwiseMetric(
    metric="pairwise_ground_truth_quality",
    metric_prompt_template=pairwise_model_compare_metric_prompt,
)

eval_task = EvalTask(dataset=pairwise_df,
                    metrics=[pairwise_ground_truth_text_quality],
                    experiment="pairwise-eval")
result_pairwise = eval_task.evaluate()
result_pairwise.metrics_table.rename(columns={"baseline_model_response": "Response A", "response": "Response B"}).replace(['BASELINE', 'CANDIDATE'], ['A', 'B'])

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/pairwise-eval-9791b75c-d636-4508-a6a8-ca93cdfdec4f to Experiment: pairwise-eval


INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 3 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 3/3 [00:06<00:00,  2.08s/it]
INFO:vertexai.evaluation._evaluation:All 3 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:6.244302168000047 seconds


,prompt,reference,Response B,Response A,pairwise_ground_truth_quality/explanation,pairwise_ground_truth_quality/pairwise_choice
0,This account is not protected with a two-factor authentication.,2차 인증이 되지 않은 계정입니다.,이중 인증으로 보호되지 않은 계정입니다.,이 계정은 이중 인증으로 안전하게 보호되지 않습니다.,"1. 예시 일치성: Response B는 Ground Truth와 매우 유사하게 번역되었습니다. Response A는 '안전하게 보호되지 않습니다'라는 부분이 Ground Truth와 다릅니다. 2. 지시를 잘 따르는지: Response A와 B 모두 자연스럽고 명확하게 번역되었지만, Response B가 Ground Truth의 간결한 표현을 더 잘 따랐습니다.",B
1,Even the longest journey begins with a single step.,천 리 길도 한 걸음부터.,가장 먼 여정도 한 걸음에서 시작됩니다.,가장 긴 여행조차도 한 걸음부터 시작됩니다.,"Response B는 '먼 여정'으로 번역하여 Ground Truth의 의미에 더 가깝습니다. Response A의 '긴 여정'은 Ground Truth가 의도하는 속담의 뉘앙스를 제대로 전달하지 못합니다. 둘 다 자연스럽지만, Response B가 Ground Truth의 '천 리 길'이라는 비유적 표현에 더욱 적합한 표현을 사용했습니다.",B
2,Even the longest journey begins with a single step.,Лиха беда начало.,Даже самая длинная дорога продолжается с одного шага.,Даже самый долгий путь начинается с первого шага.,"Response A는 '가장 긴 여정도 한 걸음부터 시작된다'라는 Source 문장의 의미를 정확하게 반영한 번역입니다. Response B는 '가장 긴 길도 한 걸음으로 계속된다'로 번역하여 Source 문장과 Ground Truth 모두의 의미와 어긋납니다. Ground Truth인 'Лиха беда начало'는 '시작이 반이다' 또는 '어려운 일도 시작이 어려울 뿐이다'라는 러시아 속담으로, Response A가 그 의미에 더 가깝습니다.",A
